In [2]:
import pandas as pd
import sklearn
import xgboost as xgb
import numpy as np

In [3]:
race_features = pd.read_pickle("data/race_features.pickle")


In [4]:
#del race_features['rc_track_course']
list(race_features.columns)

['race_index',
 'dr',
 'lbw',
 'win_odds',
 'act_wt',
 'declare_horse_wt',
 'running_position_1',
 'running_position_2',
 'running_position_3',
 'running_position_4',
 'finish_time_micro_sec',
 'rating',
 'dist_per_sec',
 'current_rating',
 'start_season_rating',
 'season_stakes',
 'total_stakes',
 'no_start_past_10',
 'age',
 'no_123_starts_1',
 'no_123_starts_2',
 'no_123_starts_3',
 'no_123_starts_4',
 'win_return',
 'win_return2',
 'jockey_win_return',
 'jockey_win_return2',
 'trainer_win_return',
 'trainer_win_return2',
 'win',
 'g_F',
 'g_FT',
 'g_G',
 'g_GD',
 'g_GF',
 'g_GY',
 'g_WF',
 'g_WS',
 'g_Y',
 'race_class_1',
 'race_class_2',
 'race_class_3',
 'race_class_3S',
 'race_class_4',
 'race_class_4R',
 'race_class_4S',
 'race_class_4YO',
 'race_class_5',
 'race_class_G1',
 'race_class_G2',
 'race_class_G3',
 'race_class_GRIFFIN',
 'race_class_HKG1',
 'race_class_HKG2',
 'race_class_HKG3',
 'race_class_MAC G1',
 'race_class_R',
 'race_class_SIN G1',
 'dist_1000',
 'dist_1200',

In [5]:
#race_features[race_features.isna().any(axis=1)]


#race_features[race_features['dr'].isna()] # drop dr na
#race_features[race_features['finish_time_micro_sec'].isna()]
#race_features[race_features['declare_horse_wt'].isna()]
#race_features.dropna().shape

#race_features.loc[:, "finish_time_micro_sec":].isna().sum()

#race_features.loc[:, ["jockey"]]
#ace_features.isna().any(axis=1).sum()
race_features = race_features.dropna()
#ace_features.shape
race_features.shape

(11704, 1237)

In [6]:
train, test = sklearn.model_selection.train_test_split(race_features, test_size=0.2)

train.drop(["win_return"], axis=1).columns.tolist()
#test.drop(test[['win', 'win_odds']], axis=1)

['race_index',
 'dr',
 'lbw',
 'win_odds',
 'act_wt',
 'declare_horse_wt',
 'running_position_1',
 'running_position_2',
 'running_position_3',
 'running_position_4',
 'finish_time_micro_sec',
 'rating',
 'dist_per_sec',
 'current_rating',
 'start_season_rating',
 'season_stakes',
 'total_stakes',
 'no_start_past_10',
 'age',
 'no_123_starts_1',
 'no_123_starts_2',
 'no_123_starts_3',
 'no_123_starts_4',
 'win_return2',
 'jockey_win_return',
 'jockey_win_return2',
 'trainer_win_return',
 'trainer_win_return2',
 'win',
 'g_F',
 'g_FT',
 'g_G',
 'g_GD',
 'g_GF',
 'g_GY',
 'g_WF',
 'g_WS',
 'g_Y',
 'race_class_1',
 'race_class_2',
 'race_class_3',
 'race_class_3S',
 'race_class_4',
 'race_class_4R',
 'race_class_4S',
 'race_class_4YO',
 'race_class_5',
 'race_class_G1',
 'race_class_G2',
 'race_class_G3',
 'race_class_GRIFFIN',
 'race_class_HKG1',
 'race_class_HKG2',
 'race_class_HKG3',
 'race_class_MAC G1',
 'race_class_R',
 'race_class_SIN G1',
 'dist_1000',
 'dist_1200',
 'dist_1400',


In [7]:
train_drop = train.drop(train[['win', 'finish_time_micro_sec', 'dist_per_sec', 'race_index', 'running_position_1', 'running_position_2', 'running_position_3', 'running_position_4']], axis=1)
gbm = xgb.XGBClassifier(objective="binary:logistic").fit(train_drop, train['win'])

test_drop = test.drop(test[['win', 'finish_time_micro_sec', 'dist_per_sec', 'race_index', 'running_position_1', 'running_position_2', 'running_position_3', 'running_position_4']], axis=1)
predictions = gbm.predict_proba(train_drop)

predictions


array([[0.41774517, 0.5822548 ],
       [0.9186809 , 0.08131909],
       [0.2286154 , 0.7713846 ],
       ...,
       [0.566864  , 0.43313596],
       [0.9958423 , 0.00415772],
       [0.9278476 , 0.07215239]], dtype=float32)

In [8]:
predictions = gbm.predict_proba(test_drop)

#res = pd.DataFrame(test[['win']])
res = test.copy()
res['pred'] = predictions[:, 0]
res['pred2'] = res[['pred']].apply(lambda x: 0 if x['pred'] > .5 else 1, axis=1)
#predictions[:, 0]
display(res[['pred2', 'win']].apply(lambda x: x['pred2'] == x['win'], axis=1).sum() /len(res))

res['pred_prof'] = res.apply(lambda x: (1-x['pred']) * x['win_odds'], axis=1)



0.9064502349423323

In [9]:
res[['pred', 'win', 'pred_prof']]

,pred,win,pred_prof
3041,0.998241,0,0.021111
11929,0.999714,0,0.010597
6315,0.997305,0,0.015090
726,0.976773,0,0.109169
725,0.995917,0,0.081657
11720,0.846180,0,0.969068
7648,0.529105,1,1.459775
8786,0.996484,0,0.094920
8882,0.997382,0,0.031422
11171,0.999705,0,0.109166


In [10]:
#test.drop(test[['win']], axis=1)
res =  res.iloc[np.random.permutation(len(res))]
df_profit = res.loc[res.groupby("race_index")['pred'].idxmin()]
investment = 0
payout = 0
for i, r in df_profit.iterrows():
    if r['pred'] <= 0.6:
        investment += 1
        #print("investment: " , investment)
        #print("inventment: ", investment, " row: ", r )
        if r['win']:
            print("investment: ", investment, " win.", r['win_odds'])
            payout += r['win_odds'] * .82
            #print("win_odds: " , r['win_odds'], " payout: " , payout)
        else:
            print("investment: ", investment)
display((payout - investment) / investment*100)

#res['pred']

investment:  1
investment:  2  win. 2.6
investment:  3
investment:  4  win. 6.1
investment:  5  win. 2.3
investment:  6
investment:  7
investment:  8  win. 2.2
investment:  9  win. 18.0
investment:  10  win. 3.1
investment:  11  win. 4.3
investment:  12  win. 13.0
investment:  13  win. 1.5
investment:  14  win. 4.4
investment:  15  win. 3.3
investment:  16  win. 6.1
investment:  17  win. 2.8
investment:  18
investment:  19  win. 1.9
investment:  20
investment:  21  win. 8.0
investment:  22
investment:  23  win. 3.2
investment:  24  win. 2.0
investment:  25  win. 2.1
investment:  26  win. 2.3
investment:  27  win. 1.3
investment:  28  win. 5.0
investment:  29  win. 1.9
investment:  30  win. 6.1
investment:  31  win. 15.0
investment:  32  win. 3.1
investment:  33  win. 2.7
investment:  34  win. 9.5
investment:  35  win. 4.9
investment:  36
investment:  37  win. 3.0
investment:  38  win. 3.7
investment:  39
investment:  40  win. 9.4
investment:  41  win. 2.6
investment:  42
investment:  4

171.98769230769224

In [11]:
#for i in res.groupby("race_index"):
#    display(i[1])
         
#test.drop(test[['win']], axis=1)

res =  res.iloc[np.random.permutation(len(res))]
df_profit = res.loc[res.groupby("race_index")['pred_prof'].idxmax()]
investment = 0
payout = 0
for i, r in df_profit.iterrows():
    if r['pred_prof'] >= 2:
        investment += 1
        #print("investment: " , investment)
        #print("inventment: ", investment, " row: ", r )
        if r['win']:
            #print("payout: ", payout, " win.", r['win_odds'])
            payout += r['win_odds'] * .82
            #print("win_odds: " , r['win_odds'], " payout: " , payout)
        #else:
            #payout -= investment
            #print("payout: ", payout)
#display(payout, investment)
display((payout - investment) / investment * 100)

#res['pred']

394.609090909091

In [24]:
#for i in res.groupby("race_index"):
#    display(i[1])
         
#test.drop(test[['win']], axis=1)

res2 =  res.iloc[np.random.permutation(len(res))].copy()
res2['pred'] = 1 - res2['pred']
#res2.set_index("race_index")
#display( res2.merge(pd.DataFrame(res2.groupby("race_index")['pred'].agg(lambda x: max((1-x) / sum(1-x)) )), how='left', on="race_index" ).sort_values("race_index"))
res2 = res2.merge(res2.groupby('race_index')['pred'].sum().to_frame(name="prob_group_sum"), how='left', left_index=False, right_index=True, left_on="race_index")
res2['pred3'] = res2.apply(lambda x: x['pred']/x['prob_group_sum'] , axis=1)
res2['pred3_adj_odds'] = 1 / res2['pred3']
res2['pred3_prof'] = res2.apply(lambda x: x['win_odds'] * x['pred3'], axis=1)

#display(res2)
#display(res2)
#display(pd.DataFrame(res2.groupby("race_index")['pred'].sum(), columns=["prob_group_sum"]))

"""
pred_idx = []
for r in res.groupby('race_index'):
    #display(r[1]['pred'])
    prob = 1 - r[1]['pred']
    prob2 = prob / sum(prob)
    expected_return = r[1]['win_odds'] * prob2  # not used
    #display(max(expected_return))
    if max(prob2) > .7: # filter by win probabilty
        pred_idx.append(expected_return.idxmax())
df_profit = res.loc[pred_idx]
"""
df_profit = res2.loc[res2.groupby("race_index")['pred3_prof'].idxmax()]
df_profit["pred3_bet"] = df_profit["pred3_prof"] > df_profit["win_odds"]
display(df_profit[['win', 'win_odds', 'pred', 'pred2', 'pred_prof', 'pred3', "pred3_adj_odds", 'pred3_prof', 'pred3_bet']])
display(df_profit['pred3_bet'])
investment = 0
payout = 0
for i, r in df_profit.iterrows():
    if r['pred3_prof'] >= 2: # filter 
        investment += 1
        #print("investment: " , investment)
        #print("inventment: ", investment, " row: ", r )
        if r['win'] == 1:
            print("investment: ", investment, " win.", r['win_odds'])
            payout += r['win_odds'] * .82
            #print("win_odds: " , r['win_odds'], " payout: " , payout)
        else:
            print("investment: ", investment)
display((payout) / investment*100) # the hk odds don't subtract investment

#res['pred']

,win,win_odds,pred,pred2,pred_prof,pred3,pred3_adj_odds,pred3_prof,pred3_bet
4218,0,6.5,0.395228,0,2.568982,0.535915,1.865968,3.483446,False
10396,0,68.0,0.021464,0,1.459580,0.540690,1.849489,36.766914,False
547,0,7.7,0.436594,0,3.361773,0.832655,1.200977,6.411445,False
11473,0,3.4,0.213448,0,0.725724,1.000000,1.000000,3.400000,False
9895,0,29.0,0.004607,0,0.133600,0.180845,5.529583,5.244518,False
8858,0,36.0,0.003604,0,0.129755,0.289928,3.449132,10.437409,False
6724,0,12.0,0.039732,0,0.476780,1.000000,1.000000,12.000000,False
7274,0,2.0,0.090213,0,0.180426,0.960528,1.041094,1.921056,False
11672,1,11.0,0.193187,0,2.125058,0.449183,2.226265,4.941012,False
8350,0,4.7,0.252013,0,1.184459,0.980853,1.019521,4.610007,False


4218     False
10396    False
547      False
11473    False
9895     False
8858     False
6724     False
7274     False
11672    False
8350     False
4956     False
3099     False
6819     False
9378     False
9141     False
4272     False
1334     False
1218     False
8812     False
1233     False
10447    False
6467     False
5882     False
3081     False
7633     False
122      False
9429     False
5067     False
10210    False
6600     False
         ...  
55       False
5467     False
533      False
798      False
9557     False
5014     False
9982     False
9440     False
9195     False
2396     False
4847     False
4186     False
1157     False
3599     False
2050     False
3748     False
3676     False
2956     False
3507     False
6796     False
4601     False
8840     False
8619     False
957      False
2518     False
2939     False
10483    False
2014     False
3351     False
683      False
Name: pred3_bet, Length: 757, dtype: bool

investment:  1
investment:  2
investment:  3
investment:  4
investment:  5
investment:  6
investment:  7
investment:  8  win. 11.0
investment:  9
investment:  10  win. 2.6
investment:  11
investment:  12
investment:  13  win. 94.0
investment:  14
investment:  15  win. 6.1
investment:  16
investment:  17
investment:  18
investment:  19
investment:  20
investment:  21
investment:  22  win. 2.2
investment:  23
investment:  24
investment:  25
investment:  26
investment:  27
investment:  28
investment:  29
investment:  30
investment:  31  win. 14.0
investment:  32
investment:  33  win. 18.0
investment:  34
investment:  35  win. 9.7
investment:  36
investment:  37
investment:  38
investment:  39  win. 11.0
investment:  40
investment:  41
investment:  42
investment:  43
investment:  44
investment:  45
investment:  46  win. 58.0
investment:  47
investment:  48  win. 4.3
investment:  49  win. 3.0
investment:  50  win. 13.0
investment:  51
investment:  52
investment:  53  win. 5.6
investment:  5

investment:  556  win. 3.2
investment:  557
investment:  558
investment:  559  win. 3.8
investment:  560
investment:  561
investment:  562  win. 4.0
investment:  563  win. 5.5
investment:  564
investment:  565
investment:  566
investment:  567  win. 2.8
investment:  568
investment:  569
investment:  570
investment:  571
investment:  572
investment:  573
investment:  574  win. 2.1
investment:  575
investment:  576
investment:  577
investment:  578
investment:  579
investment:  580  win. 14.0
investment:  581
investment:  582
investment:  583
investment:  584
investment:  585
investment:  586
investment:  587
investment:  588
investment:  589
investment:  590
investment:  591
investment:  592
investment:  593
investment:  594
investment:  595
investment:  596
investment:  597
investment:  598  win. 2.8
investment:  599
investment:  600
investment:  601
investment:  602
investment:  603
investment:  604
investment:  605
investment:  606
investment:  607
investment:  608
investment:  609
i

207.40284495021353

In [ ]:
# do train test split by date   